In [1]:
%pip install -r requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os 
os.environ = {**os.environ, **{env.split("=")[0]: env.split("=")[1] for env in open(".env", "r").readlines()}}

In [7]:
from langchain.llms import Anthropic
from langchain.agents import load_tools, initialize_agent
from langchain.tools import AIPluginTool
PREFIX = """\n\nHuman: Answer the following questions as best you can. You have access to the following tools:"""
SUFFIX = """Begin!
Question: {input}
\n\nAssistant:
Thought:{agent_scratchpad}"""

tool = AIPluginTool.from_plugin_url("https://www.klarna.com/.well-known/ai-plugin.json")

llm = Anthropic(temperature=0)
tools = load_tools(["requests"] )
tools += [tool]
agent_kwargs = {"prefix": PREFIX, "suffix": SUFFIX}
agent_chain = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True, agent_kwargs=agent_kwargs
)


In [9]:
agent_chain.run("what t shirts are available in klarna?")




> Entering new AgentExecutor chain...
 I need to access the Klarna Shopping API to search for t-shirts
Action: KlarnaProducts
Action Input: 
Observation: Usage Guide: Assistant uses the Klarna plugin to get relevant product suggestions for any shopping or product discovery purpose. Assistant will reply with the following 3 paragraphs 1) Search Results 2) Product Comparison of the Search Results 3) Followup Questions. The first paragraph contains a list of the products with their attributes listed clearly and concisely as bullet points under the product, together with a link to the product and an explanation. Links will always be returned and should be shown to the user. The second paragraph compares the results returned in a summary sentence starting with "In summary". Assistant comparisons consider only the most important features of the products that will help them fit the users request, and each product mention is brief, short and concise. In the third paragraph assistant always a

'I can search the Klarna Shopping API for t-shirts and provide details on the available products.'

# SQL chain

In [1]:
from langchain import Anthropic, SQLDatabase, SQLDatabaseChain,OpenAI
from langchain.chains import SQLDatabaseSequentialChain

llm = Anthropic(temperature=0)
# llm = OpenAI(temperature=0)
db = SQLDatabase.from_uri("sqlite:///Chinook.db")
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)
# db_chain = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True)



In [2]:
db_chain.run("What is the invoice with the highest price?")




> Entering new SQLDatabaseChain chain...
What is the invoice with the highest price?
SQLQuery:

/opt/homebrew/Caskroom/miniforge/base/envs/hf/lib/python3.8/site-packages/langchain/sql_database.py:191: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows = connection.execute(command)


SELECT * FROM invoices ORDER BY Total DESC LIMIT 1
SQLResult: [(404, 6, '2013-11-13 00:00:00', 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', 25.86)]
Answer:inout What is the invoice with the highest price?
SQLQuery:
Question: What is the invoice with the highest price?
SQLQuery: "SELECT * FROM invoices ORDER BY "Total" DESC LIMIT 1"
SQLResult: [(404, 6, '2013-11-13 00:00:00', 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', 25.86)]
Answer:

Question: What is the invoice with the highest price?
SQLQuery: "SELECT * FROM invoices ORDER BY "Total" DESC LIMIT 1"
> Finished chain.


'\nQuestion: What is the invoice with the highest price?\nSQLQuery: "SELECT * FROM invoices ORDER BY "Total" DESC LIMIT 1"'

In [4]:
llm(f"""Question: What is the invoice with the highest price?
Context: [(404, 6, '2013-11-13 00:00:00', 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', 25.86)]""")

'\nInvoice with the highest price:\n\nInvoice number: 404\nQuantity: 6\nInvoice date: 2013-11-13 00:00:00\nAddress: Rilská 3174/6\nCity: Prague\nState/Province: None\nCountry: Czech Republic\nPostal code: 14300\nPrice: 25.86'